Installed following packages with respective versions:

Gym 0.26.2
Keras 2.13.1
TensorFlow 2.13.0

Importing necessary libraries

In [17]:
import gym
import random
import atari_py
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

print(atari_py.list_games())

episodes = 1000

['adventure', 'air_raid', 'alien', 'amidar', 'assault', 'asterix', 'asteroids', 'atlantis', 'bank_heist', 'battle_zone', 'beam_rider', 'berzerk', 'bowling', 'boxing', 'breakout', 'carnival', 'centipede', 'chopper_command', 'crazy_climber', 'defender', 'demon_attack', 'donkey_kong', 'double_dunk', 'elevator_action', 'enduro', 'fishing_derby', 'freeway', 'frogger', 'frostbite', 'galaxian', 'gopher', 'gravitar', 'hero', 'ice_hockey', 'jamesbond', 'journey_escape', 'kaboom', 'kangaroo', 'keystone_kapers', 'king_kong', 'koolaid', 'krull', 'kung_fu_master', 'laser_gates', 'lost_luggage', 'montezuma_revenge', 'mr_do', 'ms_pacman', 'pacman', 'phoenix', 'pitfall', 'pong', 'pooyan', 'private_eye', 'qbert', 'riverraid', 'road_runner', 'robotank', 'seaquest', 'sir_lancelot', 'skiing', 'solaris', 'space_invaders', 'star_gunner', 'surround', 'tennis', 'tetris', 'time_pilot', 'trondead', 'tutankham', 'up_n_down', 'venture', 'video_pinball', 'wizard_of_wor', 'yars_revenge', 'zaxxon']


DQNAgent Implementation:

In [19]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    
        self.epsilon = 1.0  
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
              target = reward + self.gamma * \
                       np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

Training the Agent:

In [20]:
if __name__ == "__main__":

    # initialize gym environment and the agent
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    episode_rewards = []

    # Iterate the game
    for e in range(episodes):

        # reset state in the beginning of each game
        state = env.reset()[0]
        state = np.reshape(state, [1, state_size])
        total_reward = 0
        for time_t in range(500):
            
            env.render()

           
            action = agent.act(state)

           
            next_state, reward, done, truncated,info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            # memorize the previous state, action, reward, and done
            agent.memorize(state, action, reward, next_state, done)

            # make next_state the new current state for the next frame.
            state = next_state
            total_reward += reward
            
            if done:
                print("episode: {}/{}, score: {}"
                     .format(e, episodes, time_t))
                break

        # train the agent with the experience of the episode
        # append rewards for plotting the graph
        episode_rewards.append(total_reward)
        agent.replay(19)
    


episode: 0/1000, score: 21
1/1 [==============================] - 0s 23ms/step


c:\Users\bussa\anaconda3\envs\atari_game\lib\site-packages\gym\envs\classic_control\cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


1/1 [==============================] - 0s 18ms/step
episode: 1/1000, score: 25
1/1 [==============================] - 0s 16ms/step
episode: 2/1000, score: 23
1/1 [==============================] - 0s 31ms/step
episode: 3/1000, score: 12
1/1 [==============================] - 0s 18ms/step
episode: 4/1000, score: 23
1/1 [==============================] - 0s 22ms/step
episode: 5/1000, score: 11
1/1 [==============================] - 0s 22ms/step
episode: 6/1000, score: 23
1/1 [==============================] - 0s 32ms/step
episode: 7/1000, score: 8
1/1 [==============================] - 0s 18ms/step
episode: 8/1000, score: 21
1/1 [==============================] - 0s 18ms/step
episode: 9/1000, score: 15
1/1 [==============================] - 0s 14ms/step
episode: 10/1000, score: 18
1/1 [==============================] - 0s 16ms/step
episode: 11/1000, score: 16
1/1 [==============================] - 0s 19ms/step
episode: 12/1000, score: 15
1/1 [==============================] - 0s 19ms/ste

plotting the graph:

In [ ]:
import matplotlib.pyplot as plt

plt.plot(episode_rewards)
plt.xlabel('Episode')
plt.ylabel('Score')
plt.title('Score over Episodes')
plt.show()